# Different Ways to Faltten Deeply Nested Jsons into pandas DataFrame

Python have few modules to read and flatten json object and convert it into Pandas DataFrame
- `Json_normalize`
-  `read_json`
- `json_loads`

In Few Cases Even after using all modules the json is still nested. In this notebook we will see few methos to do it. Lets' Get Started

## Import Python Modules

In [36]:
import json
import pandas as pd

We will try to read a simple json and convert into dataframe

In [3]:
sample_json = {'Name':'Joey', 'Address':{'City':'Bangalore','State':'KA'}, 'Country':'India'}
sample_json

{'Name': 'Joey',
 'Address': {'City': 'Bangalore', 'State': 'KA'},
 'Country': 'India'}

In [8]:
df = pd.json_normalize(sample_json)
df

,Name,Country,Address.City,Address.State
0,Joey,India,Bangalore,KA


Looks good!! This is how `json_normalize` can be used to flatten semi-structured jsons. But this alone can't be used to flatten deeply nested Jsons. see below scenario as an example

**Raw_Data**

In [45]:
{
   "_id": {
      "emp_id": "123456",
      "DOJ": "20210101"
   },
   "emp": {
      "details": {
         "WorkLocation": {
            "Country": "India"
         },
         "body": {
            "Document": {
               "action": "Add",
               "id": "123456",
               "Doc": {
                  "Identification": {
                     "FirstName": "Joey",
                     "Last Name": "Lark",
                     "Workstation": "FL03"
                  },
                  "Dates": [
                     {
                        "DOJ": "20210101"
                     },
                     {
                        "DOP": "20210601"
                     }
                  ],
                  "Status": {
                     "Position": "Active",
                     "StsFlag": "1"
                  },
                  "Address": {
                     "CurrentAddress": [
                        {
                           "CurrentOwner": {
                              "Name": "Test Building, Bangalore",
                              "Pincode": "345678"
                           }
                        }
                     ]
                  }
               }
            }
         }
      }
   }
}


{'_id': {'emp_id': '123456', 'DOJ': '20210101'},
 'emp': {'details': {'WorkLocation': {'Country': 'India'},
   'body': {'Document': {'action': 'Add',
     'id': '123456',
     'Doc': {'Identification': {'FirstName': 'Joey',
       'Last Name': 'Lark',
       'Workstation': 'FL03'},
      'Dates': [{'DOJ': '20210101'}, {'DOP': '20210601'}],
      'Status': {'Position': 'Active', 'StsFlag': '1'},
      'Address': {'CurrentAddress': [{'CurrentOwner': {'Name': 'Test Building, Bangalore',
          'Pincode': '345678'}}]}}}}}}}

In [60]:
with open('nested_json.json','r') as f:
    data = json.load(f)

df = pd.json_normalize(data)
df

,_id.emp_id,_id.DOJ,emp.details.WorkLocation.Country,emp.details.body.Document.action,emp.details.body.Document.id,emp.details.body.Document.Doc.Identification.FirstName,emp.details.body.Document.Doc.Identification.Last Name,emp.details.body.Document.Doc.Identification.Workstation,emp.details.body.Document.Doc.Dates,emp.details.body.Document.Doc.Status.Position,emp.details.body.Document.Doc.Status.StsFlag,emp.details.body.Document.Doc.Address.CurrentAddress
0,123456,20210101,India,Add,123456,Joey,Lark,FL03,"[{'DOJ': '20210101'}, {'DOP': '20210601'}]",Active,1,"[{'CurrentOwner': {'Name': 'Test Building, Ban..."


In [57]:
df.columns

Index(['_id.emp_id', '_id.DOJ', 'emp.details.WorkLocation.Country',
       'emp.details.body.Document.action', 'emp.details.body.Document.id',
       'emp.details.body.Document.Doc.Identification.FirstName',
       'emp.details.body.Document.Doc.Identification.Last Name',
       'emp.details.body.Document.Doc.Identification.Workstation',
       'emp.details.body.Document.Doc.Dates',
       'emp.details.body.Document.Doc.Status.Position',
       'emp.details.body.Document.Doc.Status.StsFlag',
       'emp.details.body.Document.Doc.Address.CurrentAddress'],
      dtype='object')

Here We can see for below two columns even after using `json_normalize` the json isn't flattened
- emp.details.body.Document.Doc.Dates
- emp.details.body.Document.Doc.Address.CurrentAddress

Let's see Few methods on how can we still flatten this json

I have written below Function which can flatten nested column in a dataframe

In [61]:
def flat_nested_cols(alist):
        """
        Pass a List of Nested Columns and it returns flattened columns
        Args:
        INPUT - alist- Pass a List which have nested dictionaries inside it
        OUTPUT - Returns a Dictionary which have seperate Header and Values for Each Element inside a nested dictionary
        """
        outputdict = {}
        for dic in alist:
            for key, value in dic.items():
                if isinstance(value, dict):
                    for k2, v2, in value.items():
                        #Append Key as a prefix to Each Header Name
                        k2=key+'.'+k2
                        outputdict[k2] = outputdict.get(k2, []) + [v2]
                else:
                    outputdict[key] = outputdict.get(key, []) + [value]
        return outputdict 

Making use of Above Function. FLatten the Two columns from above dataframe

In [62]:
cols_list = ['emp.details.body.Document.Doc.Dates',
            'emp.details.body.Document.Doc.Address.CurrentAddress']

#Iterate this loop for each column in the Cols_list
for col in cols_list:
    li=[]
    #Ti Iterdate through each row in a dataframe
    for i in range(len(df['_id.emp_id'])):
        #Using Try and Expect Method in order to avoid nulls
        try:
            a=flat_nested_cols(df[col][i])
            li.append(a)
        except:
            li.append({})
    #Load the List into a dataframe
    df_l = pd.DataFrame(li)
    #Concat with the Main DataFrame
    df = pd.concat([df.reset_index(drop=True),df_l.reset_index(drop=True)], axis=1)

#Drop the columns in Cols_list
df.drop(cols_list, axis=1, inplace=True)


In [63]:
df.columns

Index(['_id.emp_id', '_id.DOJ', 'emp.details.WorkLocation.Country',
       'emp.details.body.Document.action', 'emp.details.body.Document.id',
       'emp.details.body.Document.Doc.Identification.FirstName',
       'emp.details.body.Document.Doc.Identification.Last Name',
       'emp.details.body.Document.Doc.Identification.Workstation',
       'emp.details.body.Document.Doc.Status.Position',
       'emp.details.body.Document.Doc.Status.StsFlag', 'DOJ', 'DOP',
       'CurrentOwner.Name', 'CurrentOwner.Pincode'],
      dtype='object')

In [64]:
df

,_id.emp_id,_id.DOJ,emp.details.WorkLocation.Country,emp.details.body.Document.action,emp.details.body.Document.id,emp.details.body.Document.Doc.Identification.FirstName,emp.details.body.Document.Doc.Identification.Last Name,emp.details.body.Document.Doc.Identification.Workstation,emp.details.body.Document.Doc.Status.Position,emp.details.body.Document.Doc.Status.StsFlag,DOJ,DOP,CurrentOwner.Name,CurrentOwner.Pincode
0,123456,20210101,India,Add,123456,Joey,Lark,FL03,Active,1,[20210101],[20210601],"[Test Building, Bangalore]",[345678]


Now we can see the nested columns are flattened. After trying various ways and methods. I found this to be easy and simplest. I hope this helps :)